# COVID-19: United States Confirmed Cases & Deaths 

The following data cleaning and merging is performed to load the data into Power BI for further visualization and analysis.

In [20]:
# Packages / libraries
import os
import numpy as np
import pandas as pd
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

The data collected for this analysis is operated by the Johns Hopkins University Center for Systems Science and Engineering (https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)

In [21]:
# load raw data from https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
confirmed = pd.read_csv("C:/Users/garcr/Desktop/time_series_covid19_confirmed_US.csv")
deaths = pd.read_csv("C:/Users/garcr/Desktop/time_series_covid19_deaths_US.csv")

# Confirm that the dataframes have the same number of rows (countries) and columns (dates)
# Deaths dataframe has an extra column 'Population'
print('The shape of confirmed is:', confirmed.shape)
print('The shape of deaths is:', deaths.shape)

confirmed.head()
#deaths.head()

The shape of confirmed is: (3342, 485)
The shape of deaths is: (3342, 486)


,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/30/21,5/1/21,5/2/21,5/3/21,5/4/21,5/5/21,5/6/21,5/7/21,5/8/21,5/9/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6904,6907,6909,6910,6910,6914,6914,6918,6918,6920
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20941,20966,20983,20993,20993,21035,21093,21107,21123,21131
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2300,2302,2302,2302,2302,2307,2307,2307,2307,2308
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,2594,2596,2596,2597,2597,2604,2604,2604,2605,2607
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,6613,6616,6619,6621,6621,6635,6645,6651,6656,6660


In [22]:
# Drop unnecessary columns
confirmed.drop(['iso2','iso3','code3','FIPS','Combined_Key'], axis=1, inplace=True)
deaths.drop(['iso2','iso3','code3','FIPS','Combined_Key'], axis=1, inplace=True)

#confirmed.head()
#deaths.head()

In [23]:
# Melt the date columns into one column
confirmed2 = pd.melt(confirmed, id_vars=[
                     'UID', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_'], var_name=['Date'])
deaths2 = pd.melt(deaths, id_vars=['UID', 'Admin2', 'Province_State',
                                   'Country_Region', 'Lat', 'Long_', 'Population'], var_name=['Date'])

print('The shape of confirmed is:', confirmed2.shape)
print('The shape of deaths is:', deaths2.shape)

# confirmed2.head()

The shape of confirmed is: (1584108, 8)
The shape of deaths is: (1584108, 9)


In [24]:
# Convert Date column into datetime objects (xxxx-xx-xx)
confirmed2['Date'] = pd.to_datetime(confirmed2['Date'])
deaths2['Date'] = pd.to_datetime(deaths2['Date'])

In [25]:
# Replace column headings with detailed fields
confirmed2.columns = confirmed2.columns.str.replace('value', 'Confirmed')
confirmed2.columns = confirmed2.columns.str.replace('Admin2', 'County')
deaths2.columns = deaths2.columns.str.replace('value', 'Deaths')
deaths2.columns = deaths2.columns.str.replace('Admin2', 'County')

# Investigate NULLs before join
print(confirmed2.isnull().sum())
print(deaths2.isnull().sum())

UID                  0
County            2844
Province_State       0
Country_Region       0
Lat                  0
Long_                0
Date                 0
Confirmed            0
dtype: int64
UID                  0
County            2844
Province_State       0
Country_Region       0
Lat                  0
Long_                0
Population           0
Date                 0
Deaths               0
dtype: int64


The null county values are due to the the District of Columbia, the 5 inhabited US territories (American Samoa, Guam, Northern Mariana Islands, Puerto Rico, US Virgin Islands) and the Grand Princess and Diamond Princess cruise ships having no counties. The null county value will be filled with the 'Province_State' name of the corresponding territory or cruise ship. 

In [26]:
# Join the dataframes
covid = confirmed2.merge(deaths2[['UID', 'Province_State', 'Country_Region', 'Date', 'Deaths', 'Population']],
                         how='outer',
                         left_on=['UID', 'Province_State',
                                  'Country_Region', 'Date'],
                         right_on=['UID', 'Province_State', 'Country_Region', 'Date'])

# Investigate the shape of the dataframe after the join
print('\nThe shape of confirmed is:', confirmed2.shape)
print('The shape of deaths is:', deaths2.shape)
print('The shape of the joined dataframe is:', covid.shape)

# print(covid.isnull().sum())
# covid.tail()


The shape of confirmed is: (1584108, 8)
The shape of deaths is: (1584108, 9)
The shape of the joined dataframe is: (1584108, 10)


The joined dataframe "covid" has 10 columns because Deaths and Population has been added to the confirmed dataframe. 

In [27]:
# Fill County NaN values with Province_State values (Applies only to DC, US territories, and cruise ships)
covid['County'].fillna(covid['Province_State'], inplace=True)

# covid.head()
covid.isnull().sum()

UID               0
County            0
Province_State    0
Country_Region    0
Lat               0
Long_             0
Date              0
Confirmed         0
Deaths            0
Population        0
dtype: int64

In [28]:
# Add Month-Year column
covid['Month-Year'] = covid['Date'].dt.strftime('%b-%Y')
# covid.tail()

In [29]:
# Copy df
temp = covid.copy()

# Create columns for previous date's COVID-19 cases to create daily aggregates and a running total
temp['Current Date'] = covid['Date'] + pd.Timedelta(days=1)
temp.rename(columns={'Confirmed': 'Confirmed - 1',
                     'Deaths': 'Deaths - 1', 'Date': 'Date - 1'}, inplace=True)

# Perform left join on DFs
covid_19 = covid.merge(temp[['UID', 'Province_State', 'Country_Region', 'Confirmed - 1', 'Deaths - 1',
                             'Current Date', 'Date - 1']], how='left',
                       left_on=['UID', 'Province_State',
                                'Country_Region', 'Date'],
                       right_on=['UID', 'Province_State', 'Country_Region', 'Current Date'])
print(covid_19.shape)
# covid_19.head()

(1584108, 15)


In [30]:
# Calculate the daily numbers for confirmed cases & deaths (current aggregate - previous aggregate = daily confirmed)
covid_19['Daily Confirmed'] = covid_19['Confirmed'] - covid_19['Confirmed - 1']
covid_19['Daily Deaths'] = covid_19['Deaths'] - covid_19['Deaths - 1']

print(covid_19.shape)
# covid_19.head()

(1584108, 17)


In [31]:
# Include daily numbers for the first day of data where there is no previous date available (2020-01-22)
covid_19['Daily Confirmed'].loc[covid_19['Date'] == '2020-01-22'] = covid_19['Confirmed']
covid_19['Daily Deaths'].loc[covid_19['Date'] == '2020-01-22'] = covid_19['Deaths']

# Delete unnecessary columns
covid_19.drop(['Confirmed - 1', 'Deaths - 1', 'Current Date', 'Date - 1'], axis=1, inplace=True)
print(covid_19.shape)
#covid_19.head()

(1584108, 13)


In [32]:
# Add Week Number column 
covid_19['Week Number'] = covid_19['Date'].dt.strftime('%Y-%W')

# Calculate weekly sum by state (Week is Monday-Sunday)
weekly_covid = covid_19.groupby(['Province_State', 'Week Number']).agg(
    {'Daily Confirmed': 'sum', 'Daily Deaths': 'sum'}).reset_index()

# Rename 'Daily' metrics to 'Weekly' metrics
weekly_covid.columns = weekly_covid.columns.str.replace(
    'Daily Confirmed', 'Weekly Confirmed')
weekly_covid.columns = weekly_covid.columns.str.replace(
    'Daily Deaths', 'Weekly Deaths')

# View most recent weekly numbers by state/territory
weekly_covid[weekly_covid['Week Number'] == weekly_covid['Week Number'].max()].head()

,Province_State,Week Number,Weekly Confirmed,Weekly Deaths
68,Alabama,2021-18,2391.0,65.0
137,Alaska,2021-18,732.0,2.0
206,American Samoa,2021-18,0.0,0.0
275,Arizona,2021-18,4903.0,65.0
344,Arkansas,2021-18,1405.0,18.0


In [33]:
# Calculate the weekly percentage change in confirmed cases and deaths for each state
weekly_change = weekly_covid.groupby(['Province_State', 'Week Number']).agg(
    {'Weekly Confirmed': 'sum', 'Weekly Deaths': 'sum'}).pct_change().reset_index()

In [34]:
# Add the weekly percentage change calculated above to the weekly_covid dataframe
weekly_covid[['Weekly Confirmed % Change', 'Weekly Deaths % Change']
             ] = weekly_change[['Weekly Confirmed', 'Weekly Deaths']]

# Replace NaN with 0 (zero percent change from prior week)
weekly_covid.replace(np.nan, 0, inplace=True)
weekly_covid.tail()

,Province_State,Week Number,Weekly Confirmed,Weekly Deaths,Weekly Confirmed % Change,Weekly Deaths % Change
3997,Wyoming,2021-14,426.0,1.0,0.062344,-0.8
3998,Wyoming,2021-15,394.0,2.0,-0.075117,1.0
3999,Wyoming,2021-16,429.0,2.0,0.088832,0.0
4000,Wyoming,2021-17,446.0,2.0,0.039627,0.0
4001,Wyoming,2021-18,481.0,3.0,0.078475,0.5


In [35]:
# Check the most recent US daily confirmed cases and deaths
daily_sum = covid_19.groupby(['Date']).sum()[['Daily Confirmed','Daily Deaths']]
#print(daily_sum.tail())

# Check the most recent US weekly confirmed cases and deaths 
weekly_sum = weekly_covid.groupby('Week Number').sum()[['Weekly Confirmed','Weekly Deaths']]
print(weekly_sum.tail())

             Weekly Confirmed  Weekly Deaths
Week Number                                 
2021-14              488752.0         6886.0
2021-15              472154.0         5064.0
2021-16              407147.0         4914.0
2021-17              344463.0         4705.0
2021-18              286109.0         4528.0


In [36]:
# Export Data as csv to load into Power BI
covid_19.to_csv('UScovid', sep='\t')
weekly_covid.to_csv('weekly_covid', sep='\t')